In [51]:
import pandas as pd
import joblib
from pathlib import Path
import os
import sys
import warnings
warnings.filterwarnings("ignore")
notebook_dir = os.path.dirname(os.path.abspath(''))
src_path = os.path.join(notebook_dir, '..\..\..\src')
sys.path.append(src_path)
print(src_path)

d:\ANNACONDA\projects\interest_rate_prediction\notebooks\data_cleaning\ecnomic_indicators\..\..\..\src


In [52]:
os.getcwd()

'd:\\ANNACONDA\\projects\\interest_rate_prediction\\notebooks\\data_cleaning\\ecnomic_indicators\\retail_sales'

In [53]:
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)

In [54]:
from config import RETAIL_SALES_PATH

In [55]:
retail_excel = pd.ExcelFile(f"../../../{RETAIL_SALES_PATH}retail_sales.xlsx")
retail_excel.sheet_names

['2024',
 '2023',
 '2022',
 '2021',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013',
 '2012',
 '2011',
 '2010',
 '2009',
 '2008',
 '2007',
 '2006',
 '2005',
 '2004',
 '2003',
 '2002',
 '2001',
 '2000',
 '1999',
 '1998',
 '1997',
 '1996',
 '1995',
 '1994',
 '1993',
 '1992']

In [56]:
from datetime import datetime

In [57]:
def clean_sheets():
    merged = pd.DataFrame()
    for sheet in retail_excel.sheet_names:
        print(sheet)
        retail = pd.read_excel(f"../../../{RETAIL_SALES_PATH}retail_sales.xlsx", sheet_name=sheet)
        retail = retail.iloc[3:109, 1:]
        retail["Unnamed: 1"] = retail["Unnamed: 1"].fillna("Category")
        if "Unnamed: 14" in retail.columns:
            retail = retail.drop("Unnamed: 14", axis=1)
        else:
            retail = retail.drop(["Unnamed: 5", "Unnamed: 6"], axis=1)
        retail = retail.T.reset_index(drop=True)
        retail = retail.set_index(3)
        retail = retail.T
        retail_melted = retail.melt(id_vars=["Category"], var_name="Date", value_name="Unemployment_Rate")
        retail_melted = retail_melted.dropna()
        retail_melted["Date"] = retail_melted["Date"].apply(lambda x: str(x).split("(")[0])
        retail_melted["Date"] = pd.to_datetime(retail_melted["Date"])
        merged = pd.concat([merged, retail_melted], axis=0)
    merged["Year"] = merged["Date"].dt.year
    merged["Month"] = merged["Date"].dt.month
    merged = merged.drop("Date", axis=1)
    merged["Unemployment_Rate"] = pd.to_numeric(merged["Unemployment_Rate"], errors="coerce")
    merged["Year"] = merged["Year"].astype(str)
    merged["Month"] = merged["Month"].astype(str)
    numeric_columns = merged.select_dtypes(include=["number"]).columns.tolist()
    non_numeric_columns = merged.select_dtypes(exclude=["number"]).columns.tolist()
    group_cols = ["Year", "Month", "Category"]
    retail_agg = merged[group_cols + numeric_columns].groupby(group_cols, as_index=False).sum()
    retail_pivot = retail_agg.pivot_table(index=["Year", "Month"], columns="Category", values=numeric_columns, aggfunc="sum")
    retail_pivot = retail_pivot.reset_index()
    retail_pivot.columns.name = None
    retail_pivot["Year"] = pd.to_datetime(retail_pivot["Year"], format="%Y").dt.year
    retail_pivot["Month"] = pd.to_datetime(retail_pivot["Month"], format="%m").dt.month
    retail_pivot = retail_pivot.sort_values(by=["Year", "Month"])
    

    return retail_pivot

In [58]:
retail_merged = clean_sheets()
retail_merged

2024
2023
2022
2021
2020
2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
1993
1992


Year Month                 Unemployment_Rate  \
Category             All other gen. merchandise stores   
0         1992     1                            4793.0   
4         1992     2                            4770.0   
5         1992     3                            4722.0   
6         1992     4                            4968.0   
7         1992     5                            4993.0   
..         ...   ...                               ...   
374       2023    11                           18568.0   
375       2023    12                           20297.0   
384       2024     1                           17128.0   
385       2024     2                           17370.0   
386       2024     3                           18397.0   

                                            \
Category All other home furnishings stores   
0                                      0.0   
4                                      0.0   
5                                      0.0   
6                                      0.0   
7                                      0.0   
..                                     ...   
374                                    0.0   
375                                    0.0   
384                                    0.0   
385                                    0.0   
386                                    0.0   

                                                                        \
Category Automobile and other motor vehicle dealers Automobile dealers   
0                                           56955.0            25800.0   
4                                           58660.0            27031.0   
5                                           61575.0            30195.0   
6                                           62908.0            30583.0   
7                                           63730.0            31095.0   
..                                              ...                ...   
374                                        234498.0           106976.0   
375                                        242095.0           113914.0   
384                                        226794.0           102376.0   
385                                        238377.0           109399.0   
386                                        250911.0           119987.0   

                                                                               \
Category Automotive parts, acc., and tire stores Beer, wine and liquor stores   
0                                         6334.0                       1713.0   
4                                         6374.0                       1763.0   
5                                         6897.0                       1753.0   
6                                         6950.0                       1784.0   
7                                         7087.0                       1783.0   
..                                           ...                          ...   
374                                      22526.0                       6040.0   
375                                      21834.0                       6148.0   
384                                      21898.0                       5954.0   
385                                      21592.0                       6063.0   
386                                      22983.0                       6058.0   

                                                    \
Category Beer, wine, and liquor stores Book stores   
0                               1509.0       790.0   
4                               1541.0       539.0   
5                               1597.0       535.0   
6                               1675.0       523.0   
7                               1822.0       552.0   
..                                 ...         ...   
374                             6288.0       597.0   
375                             8023.0      1073.0   
384                             5007.0       802.0   
385                             5360.0       

In [59]:
bins   = [0, 3, 6, 8, 12]
labels = ["Q1", "Q2", "Q3", "Q4"]
retail_merged["Quarter"] = pd.cut(retail_merged["Month"], bins=bins, labels=labels, right=True)
retail_merged

Year Month                 Unemployment_Rate  \
Category             All other gen. merchandise stores   
0         1992     1                            4793.0   
4         1992     2                            4770.0   
5         1992     3                            4722.0   
6         1992     4                            4968.0   
7         1992     5                            4993.0   
..         ...   ...                               ...   
374       2023    11                           18568.0   
375       2023    12                           20297.0   
384       2024     1                           17128.0   
385       2024     2                           17370.0   
386       2024     3                           18397.0   

                                            \
Category All other home furnishings stores   
0                                      0.0   
4                                      0.0   
5                                      0.0   
6                                      0.0   
7                                      0.0   
..                                     ...   
374                                    0.0   
375                                    0.0   
384                                    0.0   
385                                    0.0   
386                                    0.0   

                                                                        \
Category Automobile and other motor vehicle dealers Automobile dealers   
0                                           56955.0            25800.0   
4                                           58660.0            27031.0   
5                                           61575.0            30195.0   
6                                           62908.0            30583.0   
7                                           63730.0            31095.0   
..                                              ...                ...   
374                                        234498.0           106976.0   
375                                        242095.0           113914.0   
384                                        226794.0           102376.0   
385                                        238377.0           109399.0   
386                                        250911.0           119987.0   

                                                                               \
Category Automotive parts, acc., and tire stores Beer, wine and liquor stores   
0                                         6334.0                       1713.0   
4                                         6374.0                       1763.0   
5                                         6897.0                       1753.0   
6                                         6950.0                       1784.0   
7                                         7087.0                       1783.0   
..                                           ...                          ...   
374                                      22526.0                       6040.0   
375                                      21834.0                       6148.0   
384                                      21898.0                       5954.0   
385                                      21592.0                       6063.0   
386                                      22983.0                       6058.0   

                                                    \
Category Beer, wine, and liquor stores Book stores   
0                               1509.0       790.0   
4                               1541.0       539.0   
5                               1597.0       535.0   
6                               1675.0       523.0   
7                               1822.0       552.0   
..                                 ...         ...   
374                             6288.0       597.0   
375                             8023.0      1073.0   
384                             5007.0       802.0   
385                             5360.0       

In [60]:
retail_merged = pd.read_csv("../../../../data/csv/better_retail_sales.csv")
retail_merged

,Category,Year,Month,All other gen. merchandise stores,All other home furnishings stores,Automobile and other motor vehicle dealers,Automobile dealers,"Automotive parts, acc., and tire stores","Beer, wine and liquor stores","Beer, wine, and liquor stores",Book stores,Building mat. and garden equip. and supplies dealers,Building mat. and supplies dealers,Clothing and clothing access. stores,Clothing stores,Department stores,Department stores(excl. discount department stores),Discount dept. stores,Drinking places,Electronic shopping and mail order houses,Electronic shopping and mail-order houses,Electronics and appliance stores,Electronics stores,Family clothing stores,Floor covering stores,Food and beverage stores,Food services and drinking places,Fuel dealers,Full service restaurants,Furniture and home furnishings stores,Furniture stores,"Furniture, home furn, electronics, and appliance stores",GAFO(1),Gasoline stations,General merchandise stores,"Gift, novelty, and souvenir stores",Grocery stores,Hardware stores,Health and personal care stores,"Hobby, toy, and game stores",Home furnishings stores,Household appliance stores,Jewelry stores,Limited service eating places,Men's clothing stores,Miscellaneous store retailers,Miscellaneous stores retailers,Motor vehicle and parts dealers,New car dealers,Nonstore retailers,Office supplies and stationery stores,"Office supplies, stationery, and gift stores",Other clothing stores,Other general merchandise stores,Paint and wallpaper stores,Pharmacies and drug stores,Restaurants and other eating places,"Retail and food services sales, total",Retail sales and food services excl gasoline stations,Retail sales and food services excl motor vehicle and parts,Retail sales and food services excl motor vehicle and parts and gasoline stations,"Retail sales, total","Retail sales, total (excl. motor vehicle and parts dealers)",Shoe stores,Sporting goods stores,"Sporting goods, hobby, musical instrument, and book stores",Supermarkets and other grocery (except convenience) stores,Used car dealers,Used merchandise stores,Warehouse clubs and superstores,Women's clothing stores,Quarter
0,0,1992,1,4793,0,56955,25800,6334,1713,1509,790,19809,16696,16654,11610,24440,4818,5488,1049,2668,2692,7531,0,1649,728,60026,32863,3198,6887,8100,2392,15631,76691,24902,34875,658,55141,842,14679,620,1454,601,2044,6438,1541,3642,4318,63289,24056,13375,788,1446,0,10435,0,12819,13325,310471,285569,247182,222280,277608,214319,2814,972,6408,0,1744,371,5642,4504,Q1
1,4,1992,2,4770,0,58660,27031,6374,1763,1541,539,19927,16651,17236,11950,25723,5347,5904,1026,2632,2285,7394,0,1864,731,58952,32825,2804,6937,8250,2460,15644,78538,24181,36217,798,53943,818,14575,585,1448,576,2251,6537,1504,3902,4308,65034,25041,12169,679,1477,0,10494,0,12742,13474,311292,287111,246258,222077,278467,213433,2862,1100,6322,0,1990,402,5724,4580,Q1
2,5,1992,3,4722,0,61575,30195,6897,1753,1597,535,21594,17856,18144,12745,27460,6262,6655,1074,2477,2484,7585,0,2152,851,60208,33764,2864,7245,8487,2607,16072,81974,24911,38152,714,55083,958,14856,643,1550,631,2195,7101,1586,3755,4292,68472,28018,12315,692,1406,0,10692,0,12879,14346,323057,298146,254585,229674,289293,220821,3034,1214,6522,0,2177,419,5970,4962,Q1
3,6,1992,4,4968,0,62908,30583,6950,1784,1675,523,22368,18344,19263,13636,27990,6417,7044,1031,2757,2506,7485,0,2438,899,60830,33171,2805,7000,8500,2525,15985,84198,25153,39130,801,55455,1073,15047,706,1616,643,2216,7065,1681,4148,4391,69858,27982,12349,656,1457,0,11140,0,13107,14065,328378,303225,258520,233367,295207,225349,3246,1267,6685,0,2601,393,6172,5301,Q2
4,7,1992,5,4993,0,63730,31095,7087,1783,1822,552,23104,18775,19313,13715,28920,6636,7650,1047,2874,2483,7552,0,2412,856,62395,34250,2654,7532,8606,2605,16158,85817,26319,40382,900,56870,1232,14946,660,1670,662,2393,7545,1729,4600,4356,70817,28924,12353,697,1597,0,11462,0,13002,15077,335680,309361,264863,238544,301430,230613,3072,1293,6687,0,2171,435,6469,5365,Q2
...,...,...,...,...,...,...,...,.

In [61]:
path = Path(Path.cwd()/"../../../../data")
path

WindowsPath('d:/ANNACONDA/projects/interest_rate_prediction/notebooks/data_cleaning/ecnomic_indicators/retail_sales/../../../../data')

In [62]:
data_folder = path/'parquet/retail_sales'
data_folder.mkdir(parents=True, exist_ok=True)

In [63]:
retail_merged.to_parquet(data_folder/"retail_sales.parquet")